# Técnicas de Prendizaje Automático. Aprendizaje No Supervisado.

## Práctica 1: Algoritmo K-Means, aspectos básicos.

En esta práctica se familiarizará con una implementación del algoritmo K-Means en [scikit-learn](https://scikit-learn.org/), además de analizar su comportamiento al variar el valor de $k$ de acuerdo con los criterios de evaluación para algoritmos de agrupamiento.


**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.
<br></br>
- siempre que una función o algoritmo lo permita, establezca $random\_state=const$, donde $const$ es un valor constante de su preferencia, por ejemplo $1$. Esto para garantizar replicabilidad del experimento.
<br></br>
- incluya el código requerido entre los apartados:
   - \#\<code>
   - \#\</code>

In [29]:
#@title Utilidades
#@markdown Ejecute esta casilla para definir algunas funciones necesarias en la actividad.

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def visualize_data(x, y, labels=None):

  if type(labels) == np.ndarray or type(labels) == list:
    labels = [str(e) for e in labels]

  fig = px.scatter(x=x, y=y, color=labels)


  fig.update_layout(height=300,
                    width=300,
                    showlegend=False,
                    title={'text': 'Datos',
                          'font': {'size':12},
                          'x': 0.5,
                          'xanchor': 'center'
                          }
                    )
  return fig


def visualize_clusters_k(data, models):
  rows=2
  cols=4

  keys = sorted(models.keys())

  fig = make_subplots(rows=rows, cols=cols, subplot_titles=[f'K={k}' for k in keys])

  for i, k in enumerate(keys):
    model = models[k]
    labels = np.unique(model.labels_)
    col = i%cols
    row = int((i-col) / cols)

    col+=1
    row+=1

    for label in labels:
      x = data['X1'][model.labels_==label]
      y = data['X2'][model.labels_==label]
      fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name=f'K={k} - Clúster {label}'), row=row, col=col)


  fig.update_layout(height=600,
                    width=1200,
                    showlegend=False
                    )
  return fig


def visualize_metrics(evals):
  keys = sorted(evals)

  fig = go.Figure()

  for k in keys:
    fig.add_trace(go.Bar(x=[k], y=[evals[k]], text=f'{evals[k]:.3f}'))


  fig.update_layout(height=300,
                    width=600,
                    showlegend=False,
                    title={'text': 'Evaluaciones',
                          'font': {'size':12},
                          'x': 0.5,
                          'xanchor': 'center'
                          },
                    xaxis = {
                        'title':'K',
                        'tickmode': 'linear'
                        },
                    yaxis = {
                        'range':[min(evals.values())-0.1, max(evals.values())+0.1]
                        }

                    )
  return fig

### Carga de datos y análisis exploratorio.

**a)** El primer paso consiste en obtener los datos relacionados con la tarea dejándolos en el formato adecuado.

El fichero **aggregation-noclass.csv** contiene instancias artificiales representado puntos 〈x1,x2〉en un espacio bidimensional.

| X1 | X2|
| -- |-- |
| 15.55|28.65|
| ...|...|

En la siguiente casilla, incluya el código necesario para:

  **a.1)** leer el archivo **aggregation-noclass.csv** en un dataframe de pandas. La variable con este dataframe debe llamarse **data**.
<br><br/>
  **a.2)** visualizar los datos en un gráfico de dispersión. El código necesario ya se provee.
<br><br/>
**Sugerencia:** investigar cómo leer un archivo desde Google Colab utilizando [pandas](https://pandas.pydata.org/) y cómo visualizarlos utilizando la librería de nuestra preferencia.

In [30]:
# a.1 leer datos
import pandas as pd
data = pd.read_csv('/Users/daniel/Desktop/Master IA/TÉCNICAS DE APRENDIZAJE AUTOMÁTICO/Practica/UnSupervised/aggregation-noclass.csv')

# a.2 visualizar datos
fig = visualize_data(data['X1'], data['X2'])
fig.show()

**b)** Una vez leídos los datos, se aplicará el algoritmo K-Means para obtener los agrupamientos.

En la siguiente casilla, incluya el código necesario para:
<br><br/>
  **b.1)** aplicar la implementación de K-Means en scikit-learn con $k=2$. La variable con el modelo debe llamarce **model**.
<br><br/>
  **b.2)** imprimir el valor de los centroides.
<br><br/>
  **b.3)** visualizar los datos en un gráfico de dispersión, donde el color de cada punto estará determinado por el clúster al que pertenece. El código necesario ya se provee.
<br><br/>
**Sugerencia:** investigar los diferentes parámetros y atributos de la implementación utilizada del algoritmo K-Means.

In [31]:
from sklearn.cluster import KMeans
# b.1 aplicar algoritmo

k = 2
model = KMeans(n_clusters=k, random_state=1)
model.fit(data[['X1', 'X2']])

# b.2 imprimir centroides
print(model.cluster_centers_)

# b.3 visualizar datos
fig = visualize_data(data['X1'], data['X2'], model.labels_)
fig.show()


[[31.60300752 16.81484962]
 [13.43342912 12.82490421]]


/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



**c)** En esta parte, se explorará el efecto que tiene $k$. En la siguiente casilla, incluya el código necesario para:
<br><br/>
**c.1)** aplicar K-Means para $k \in [2,10]$, conservando los diferentes modelos. Se debe obtener un modelo para cada valor de $k$. Los modelos deben guardarse en la variable **models** que es un diccionario donde la llave es $k$ y el valor es el modelo para ese valor de $k$.
<br><br/>
**c.2)** visualizar el particionamiento que produce cada modelo. El código necesario ya se provee.
<br><br/>
Empleando un criterio puramente visual y de acuerdo con las gráficas obtenidas ¿qué valor de $k$ considera más adecuado?

In [32]:
# c.1 aplicar algoritmo
models = {}

for k in range(2, 10):  # Cambiar 11 por 10
    model = KMeans(n_clusters=k, random_state=1)
    model.fit(data[['X1', 'X2']])
    models[k] = model


# c.2 visualizar datos
fig = visualize_clusters_k(data, models)
fig.show()

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/daniel/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_km

**d)** Seleccione algún criterio aplicable para determinar el valor de K que conduce al “mejor” particionamiento de los datos, tenga en cuenta que las instancias no tienen asociada ninguna clase.

En la siguiente casilla incluya el código necesario para:
<br><br/>
**d.1)** evaluar cada modelo aplicado en el apartado anterior. Las evaluaciones deben guardarse en la variable **evals** que es un diccionario donde la llave es $k$ y el valor es la métrica calculada ese valor de $k$.
<br><br/>
**d.2)** visualizar mediante un gráfico de barras los valores de la métrica para los diferentes $k$.
<br><br/>
¿Qué valor de K recomienda en este caso.? ¿Qué podría decirse sobre la estructura encontrada por el algoritmo K-Means?
<br><br/>
**Sugerencia**: investigar sobre el coeficiente Silhouette.

In [33]:
from sklearn.metrics import silhouette_score
# d.1 evaluar métrica

evals = {}
for k in models:
    model = models[k]
    labels = model.labels_
    evals[k] = silhouette_score(data[['X1', 'X2']], labels)

# d.2 visualizar evaluaciones
fig = visualize_metrics(evals)
fig.show()

Se recomienda elegir el valor de k que maximice el coeficiente Silhouette. En este caso es k=3 o k=4. 